In [1]:
!pip install apache_beam


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 33.7 M

In [14]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Load the datasets
file_path = '/content/drive/MyDrive/DM-Assignment-2-part2/winemag-data_first150k.csv'
data = pd.read_csv(file_path)
data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [20]:
class FilterIncompleteRows(beam.DoFn):
    def process(self, row):
        country = row[1]
        price = row[4]
        if country and price:
            yield row


Composite Transform

Windowing & Triggers: These are more relevant to streaming data, but for the sake of demonstration, we'll window the data into fixed-sized bundles (let's say 10,000 rows per window) and trigger after every 5,000 rows.

In [21]:
class ComputeAveragePrice(beam.PTransform):
    def expand(self, pcoll):
        return (
            pcoll
            | 'Map country and price' >> beam.Map(lambda row: (row[1], float(row[4])))
            | 'Compute mean' >> beam.CombinePerKey(beam.combiners.MeanCombineFn())
        )


In [24]:
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

# Install apache-beam.
run('pip install --quiet apache-beam')

# Copy the input file into the local file system.
#run('mkdir -p data')
#run('gsutil cp gs://dataflow-samples/shakespeare/kinglear.txt data/')

>> pip install --quiet apache-beam



In [4]:
run("echo 2984641,Emily,35,cardio,01-09-2021 >> dept_data.txt")
run("echo 945484,Riikka,86,ortho,21-07-2021 >> dept_data.txt")
run("echo 9266396,Fanny,86,ortho,03-06-2021 >> dept_data.txt")
run("echo 5247541,Urooj,35,cardio,21-08-202 >> dept_data.txt")

>> echo 2984641,Emily,35,cardio,01-09-2021 >> dept_data.txt

>> echo 945484,Riikka,86,ortho,21-07-2021 >> dept_data.txt

>> echo 9266396,Fanny,86,ortho,03-06-2021 >> dept_data.txt

>> echo 5247541,Urooj,35,cardio,21-08-202 >> dept_data.txt




#Example 1
#Patient visit count using core transforms - Map, Filter



In [6]:

import apache_beam as beam

p1 = beam.Pipeline()
visit_count = (

  p1
   |beam.io.ReadFromText('dept_data.txt')

   |beam.Map(lambda record: record.split(','))
   |beam.Filter(lambda record: record[3] == 'cardio')
   |beam.Map(lambda record: (record[1], 1))
   |beam.CombinePerKey(sum)

   |beam.io.WriteToText('out_data.txt')
)

p1.run()

**Pipeline I/O**
Pipeline I/O refers to the input and output operations within a pipeline. This is where we read data into our pipeline and output the processed data from our pipeline.


In [36]:
(
    p
    | 'Read from CSV' >> beam.io.ReadFromText('/content/drive/MyDrive/DM-Assignment-2-part2/winemag-data_first150k.csv', skip_header_lines=1)  # Input operation
    # ... [data processing steps] ...
    | 'Write results' >> beam.Map(print)  # Output operation
)

<PCollection[[36]: Write results.None] at 0x7e2f5ef4c1c0>

**Example 2
ParDo transform**

In [8]:
import apache_beam as beam

class SplitRow(beam.DoFn):

  def process(self, element):
    # return type -> list
    return  [element.split(',')]


class FilterCardioPatients(beam.DoFn):

  def process(self, element):
    if element[3] == 'cardio':
      return [element]

class PairPatients(beam.DoFn):

  def process(self, element):
    return [(element[1], 1)]

class Counting(beam.DoFn):

  def process(self, element):
    # return type -> list
    (key, values) = element
    return [(key, sum(values))]


p1 = beam.Pipeline()

attendance_count = (

   p1
    |beam.io.ReadFromText('dept_data.txt')

    |beam.ParDo(SplitRow())
    |beam.ParDo(FilterCardioPatients())
    |beam.ParDo(PairPatients())
    | 'Group ' >> beam.GroupByKey()
    | 'Sum using ParDo' >> beam.ParDo(Counting())

    |beam.io.WriteToText('parddo_output.txt')

)

p1.run()

**Example 3
composite transform**

In [9]:
import apache_beam as beam

class CustomTransform(beam.PTransform):

  def expand(self, input_coll):

    a = (
        input_coll
                       | 'Group and sum' >> beam.CombinePerKey(sum)
                       | 'count filter accounts' >> beam.Filter(filter_on_count)
                       | 'Regular accounts employee' >> beam.Map(format_output)

    )
    return a

def SplitRow(element):
    return element.split(',')


def filter_on_count(element):
  name, count = element
  if count > 1:
    return element

def format_output(element):
  name, count = element
  #return ', '.join((name.encode('ascii'),str(count),'Regular employee'))
  return ', '.join((name,str(count),'Regular Patient'))

p = beam.Pipeline()

input_collection = (
                      p
                      | "Read from text file" >> beam.io.ReadFromText('dept_data.txt')
                      | "Split rows" >> beam.Map(SplitRow)
                   )

cardio_count = (
                      input_collection
                      | 'Get all cardio patients' >> beam.Filter(lambda record: record[3] == 'cardio')
                      | 'Pair each patient with 1' >> beam.Map(lambda record: ("cardio, " +record[1], 1))
                      | 'composite cardio Patients' >> CustomTransform()
                      | 'Write results for cardio' >> beam.io.WriteToText('cardio_output.txt')
                 )

ortho_count = (
                input_collection
                | 'Get all HR dept persons' >> beam.Filter(lambda record: record[3] == 'ortho')
                | 'Pair each hr employee with 1' >> beam.Map(lambda record: ("ortho, " +record[1], 1))
                | 'composite Ortho Patients' >> CustomTransform()
                | 'Write results for Ortho' >> beam.io.WriteToText('ortho_output.txt')
           )
p.run()

**triggers**


In [ ]:
| 'Window' >> beam.WindowInto(
      FixedWindows(60),
      trigger=AfterWatermark(),  # This is where you'd specify the trigger.
      accumulation_mode=AccumulationMode.DISCARDING)